<a href="https://colab.research.google.com/github/BATUTO90/Bat1/blob/main/copia_de_batuto_asistente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import gradio as gr
import json
import csv
import io
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
import gc

# --- Limpiar memoria ---
gc.collect()
torch.cuda.empty_cache()

# --- ETNIAS PERMITIDAS ---
ETHNICITIES = ["Latina", "Colombiana", "Egipcia", "Árabe", "Canadiense", "Mexicana"]

# --- ROLES PROFESIONALES (54) ---
ROLES = [
    {"role": "Executive Secretary", "uniform": "form-fitting tailored black dress with silk blouse", "environment": "luxurious corporate office with panoramic city views", "accessories": "slim tablet and leather portfolio"},
    {"role": "Luxury Hotel Manager", "uniform": "elegant navy blue blazer and pencil skirt ensemble", "environment": "five-star hotel lobby with marble floors and crystal chandeliers", "accessories": "gold name tag and guest register"},
    {"role": "Fashion Boutique Manager", "uniform": "chic white silk blouse and high-waisted pencil skirt", "environment": "designer boutique with mirrored walls", "accessories": "measuring tape and luxury handbag"},
    {"role": "Corporate Lawyer", "uniform": "sharp charcoal gray pantsuit with satin lapels", "environment": "modern law firm with glass walls", "accessories": "leather briefcase and legal documents"},
    {"role": "Private Jet Attendant", "uniform": "tailored navy skirt suit with gold accents", "environment": "private jet cabin with leather seats", "accessories": "champagne flute and service tray"},
    {"role": "Art Gallery Curator", "uniform": "sleek black turtleneck and pencil skirt", "environment": "minimalist white gallery with abstract art", "accessories": "catalog and laser pointer"},
    {"role": "University Professor", "uniform": "elegant tweed blazer and knee-length skirt", "environment": "ivy-covered lecture hall with chalkboard", "accessories": "leather-bound book and glasses"},
    {"role": "Wine Sommelier", "uniform": "crisp white shirt and black vest with apron", "environment": "underground wine cellar with oak barrels", "accessories": "tasting glass and corkscrew"},
    {"role": "Ballet Instructor", "uniform": "black leotard with sheer wrap skirt", "environment": "sunlit dance studio with mirrored walls", "accessories": "pointe shoes and wooden barre"},
    {"role": "Yacht Stewardess", "uniform": "white nautical uniform with gold epaulettes", "environment": "luxury yacht deck at sunset", "accessories": "silver tray with tropical cocktails"},
    {"role": "Casino Dealer", "uniform": "fitted red velvet vest and bow tie", "environment": "high-stakes poker table in VIP room", "accessories": "deck of cards and chip rack"},
    {"role": "News Anchor", "uniform": "tailored red blazer and silk blouse", "environment": "professional news studio with teleprompter", "accessories": "earpiece and broadcast microphone"},
    {"role": "Elegant Maid", "uniform": "classic French maid dress with lace trim", "environment": "opulent mansion with marble staircase", "accessories": "feather duster and silver tray"},
    {"role": "Flight Attendant", "uniform": "iconic red hat and fitted dress", "environment": "first-class airplane cabin", "accessories": "safety card and beverage cart"},
    {"role": "Sensual Nun", "uniform": "black habit with subtle lace veil", "environment": "ancient stone convent at candlelight", "accessories": "rosary and leather-bound bible"},
    {"role": "Police Officer", "uniform": "fitted navy uniform with duty belt", "environment": "urban precinct with patrol car", "accessories": "badge and handcuffs"},
    {"role": "Military Officer", "uniform": "crisp dress blues with medals", "environment": "military headquarters with flags", "accessories": "beret and salute"},
    {"role": "Nurse", "uniform": "white medical dress with red cross", "environment": "private clinic with modern equipment", "accessories": "stethoscope and chart"},
    {"role": "Schoolgirl", "uniform": "pleated plaid skirt and white blouse", "environment": "private academy library", "accessories": "satchel and textbook"},
    {"role": "Fitness Instructor", "uniform": "sleek black leggings and crop top", "environment": "high-end gym with city views", "accessories": "yoga mat and water bottle"},
    {"role": "Yoga Practitioner", "uniform": "flowing white yoga pants and bralette", "environment": "rooftop at sunrise", "accessories": "meditation beads"},
    {"role": "Salsa Dancer", "uniform": "red ruffled dress with high slit", "environment": "Havana nightclub with live band", "accessories": "dance shoes and rose"},
    {"role": "Telenovela Actress", "uniform": "dramatic emerald gown with deep neckline", "environment": "mansion balcony at golden hour", "accessories": "fan and love letter"},
    {"role": "Latin Chef", "uniform": "white chef coat with rolled sleeves", "environment": "open-air kitchen with fresh herbs", "accessories": "wooden spoon and cutting board"},
    {"role": "Fiesta Organizer", "uniform": "vibrant floral dress with off-shoulder", "environment": "colorful courtyard with papel picado", "accessories": "clipboard and maracas"},
    {"role": "Latin Sommelier", "uniform": "black vest over silk blouse", "environment": "vineyard terrace at dusk", "accessories": "wine key and tasting notebook"},
    {"role": "Flamenco Guitarist", "uniform": "black lace dress with ruffled hem", "environment": "intimate tablao with wooden stage", "accessories": "Spanish guitar"},
    {"role": "Latin Diplomat", "uniform": "tailored cream pantsuit with sash", "environment": "embassy reception with chandeliers", "accessories": "diplomatic folder and brooch"},
    {"role": "Mariachi Singer", "uniform": "charro suit with silver embroidery", "environment": "plaza at fiesta with lanterns", "accessories": "microphone and sombrero"},
    {"role": "Latin Curator", "uniform": "minimalist black dress with gold belt", "environment": "pre-Columbian artifact exhibit", "accessories": "magnifying glass and catalog"},
    {"role": "Samba Instructor", "uniform": "feathered carnival costume with heels", "environment": "Rio dance studio with mirrors", "accessories": "feather fan and rhythm sticks"},
    {"role": "1960s WAC Officer", "uniform": "olive green skirt suit with cap", "environment": "vintage military office", "accessories": "typewriter and dog tags"},
    {"role": "Vietnam Era Military Nurse", "uniform": "white duty dress with cap", "environment": "field hospital tent", "accessories": "medical kit and clipboard"},
    {"role": "1980s Air Force Pilot", "uniform": "flight suit with patches", "environment": "cockpit of F-16", "accessories": "helmet and aviators"},
    {"role": "1990s Army General", "uniform": "dress blues with stars", "environment": "Pentagon briefing room", "accessories": "pointer and map"},
    {"role": "2000s Marine Corps Captain", "uniform": "desert camo with rank", "environment": "forward operating base", "accessories": "radio and compass"},
    {"role": "2010s Cyber Warfare Specialist", "uniform": "black tactical suit", "environment": "dark server room with screens", "accessories": "laptop and encrypted drive"},
    {"role": "2020s Special Forces Operator", "uniform": "multicam fatigues", "environment": "night vision urban raid", "accessories": "rifle and comms"},
    {"role": "Artemis Program Commander", "uniform": "white NASA spacesuit", "environment": "lunar habitat module", "accessories": "helmet and mission patch"},
    {"role": "1960s WAVES Ensign", "uniform": "navy blue skirt uniform", "environment": "WWII-era naval base", "accessories": "signal flags"},
    {"role": "1970s Navy Aviator", "uniform": "flight suit with squadron patch", "environment": "aircraft carrier deck", "accessories": "g-suit and logbook"},
    {"role": "1980s Submarine Officer", "uniform": "khaki service uniform", "environment": "submarine control room", "accessories": "periscope and charts"},
    {"role": "1990s Naval Intelligence Analyst", "uniform": "dress whites", "environment": "classified briefing room", "accessories": "satellite photos"},
    {"role": "2000s Destroyer Captain", "uniform": "navy dress blues", "environment": "ship bridge at sea", "accessories": "binoculars and cap"},
    {"role": "2010s SEAL Team Coordinator", "uniform": "tactical gear with trident", "environment": "night ocean insertion", "accessories": "diving mask"},
    {"role": "2020s Female Admiral", "uniform": "full dress whites with stars", "environment": "fleet review ceremony", "accessories": "sword and medals"},
    {"role": "1960s Mercury 13 Trainee", "uniform": "white jumpsuit", "environment": "centrifuge training room", "accessories": "flight helmet"},
    {"role": "1970s Apollo-Era Engineer", "uniform": "pocket protector and skirt", "environment": "Mission Control", "accessories": "slide rule and headset"},
    {"role": "1980s Space Shuttle Pilot", "uniform": "blue flight suit", "environment": "shuttle cockpit", "accessories": "checklist and gloves"},
    {"role": "1990s Hubble Telescope Scientist", "uniform": "lab coat over dress", "environment": "NASA control room with star charts", "accessories": "telescope model"},
    {"role": "2000s ISS Mission Specialist", "uniform": "blue NASA polo and shorts", "environment": "International Space Station module", "accessories": "floating tools"},
    {"role": "2010s Mars Rover Operator", "uniform": "JPL polo and khakis", "environment": "Mars yard testing facility", "accessories": "rover controller"},
    {"role": "2020s Artemis Astronaut", "uniform": "next-gen spacesuit", "environment": "lunar south pole", "accessories": "sample bag"},
    {"role": "2025 Gateway Station Commander", "uniform": "deep space command suit", "environment": "lunar orbit station", "accessories": "holographic display"}
]

# --- MOMENTOS COTIDIANOS (30) ---
EVERYDAY_MOMENTS = [
    {"scene": "Cocina matutina", "action": "agachándose a sacar algo del horno bajo", "outfit": "camiseta oversized blanca y shorts de algodón", "setting": "cocina soleada con ventana abierta", "accessories": "taza de café en la encimera, delantal colgando"},
    {"scene": "Lavandería", "action": "doblándose para sacar ropa de la secadora", "outfit": "top deportivo y leggings ajustados", "setting": "cuarto de lavado con cestas de ropa", "accessories": "cesta de ropa limpia"},
    {"scene": "Jardinería", "action": "arrodillada plantando flores", "outfit": "vestido ligero de algodón con tirantes", "setting": "jardín trasero con macetas", "accessories": "guantes de jardinería, regadera"},
    {"scene": "Yoga en casa", "action": "haciendo postura del perro boca abajo", "outfit": "leggings de yoga y top corto", "setting": "sala con esterilla y luz natural", "accessories": "bloque de yoga"},
    {"scene": "Limpieza de estanterías", "action": "estirándose en punta de pies para alcanzar un libro", "outfit": "camiseta holgada y shorts de pijama", "setting": "biblioteca personal con escalera pequeña", "accessories": "plumero"},
    {"scene": "Paseo con el perro", "action": "agachándose a atar la correa", "outfit": "vestido veraniego floreado", "setting": "parque al atardecer", "accessories": "correa, bolsa de premios"},
    {"scene": "Café en la terraza", "action": "sentada cruzando las piernas y dejando caer una servilleta", "outfit": "blusa de seda y falda plisada", "setting": "terraza con mesa de hierro", "accessories": "libro abierto, taza de té"},
    {"scene": "Subiendo escaleras", "action": "subiendo con una bolsa de compras", "outfit": "falda lápiz y blusa ajustada", "setting": "escalera de caracol en apartamento", "accessories": "bolsa de compras"},
    {"scene": "Bailando sola", "action": "girando con música en auriculares", "outfit": "vestido corto de verano", "setting": "sala con luz cálida", "accessories": "auriculares inalámbricos"},
    {"scene": "Pintando una pared", "action": "estirándose para pintar el techo", "outfit": "camiseta vieja y shorts vaqueros", "setting": "habitación en renovación", "accessories": "brocha, escalera"},
    {"scene": "Cambiando bombilla", "action": "de pie en una silla", "outfit": "camiseta larga como vestido", "setting": "cocina con lámpara colgante", "accessories": "bombilla nueva"},
    {"scene": "Buscando algo en el armario", "action": "arrodillada revisando cajones bajos", "outfit": "camisa de dormir de satén", "setting": "dormitorio con armario abierto", "accessories": "caja de joyas"},
    {"scene": "Haciendo la cama", "action": "inclinándose sobre el colchón", "outfit": "pijama de seda corto", "setting": "dormitorio con luz de mañana", "accessories": "almohadas"},
    {"scene": "Tomando el sol", "action": "ajustando la sombrilla", "outfit": "bikini bajo vestido transparente", "setting": "balcón con tumbona", "accessories": "gafas de sol"},
    {"scene": "Cargando compras del auto", "action": "doblándose para sacar bolsas del maletero", "outfit": "vestido casual de algodón", "setting": "garaje residencial", "accessories": "bolsas de supermercado"},
    {"scene": "Regando plantas altas", "action": "estirándose para alcanzar una maceta", "outfit": "top sin mangas y shorts", "setting": "invernadero casero", "accessories": "regadera"},
    {"scene": "Probándose zapatos", "action": "sentada en el suelo con una pierna levantada", "outfit": "falda corta y blusa", "setting": "tienda de zapatos", "accessories": "caja de zapatos"},
    {"scene": "Bailando en la cocina", "action": "moviendo caderas mientras cocina", "outfit": "camiseta y shorts de algodón", "setting": "cocina moderna", "accessories": "cuchara de madera"},
    {"scene": "Limpiando el piso", "action": "en cuatro puntos fregando", "outfit": "vestido ligero de verano", "setting": "sala con balde", "accessories": "trapo"},
    {"scene": "Colgando cortinas", "action": "de pie en una silla con los brazos arriba", "outfit": "blusa y falda corta", "setting": "ventana grande", "accessories": "cortinas nuevas"},
    {"scene": "Leyendo en el sofá", "action": "cruzando las piernas y dejando caer el libro", "outfit": "vestido de punto ajustado", "setting": "sala con manta", "accessories": "libro"},
    {"scene": "Haciendo estiramiento", "action": "tocando los dedos de los pies", "outfit": "leggings y top deportivo", "setting": "sala con TV", "accessories": "esterilla"},
    {"scene": "Saliendo de la ducha", "action": "secándose el cabello con toalla", "outfit": "toalla corta envuelta", "setting": "baño con espejo empañado", "accessories": "secador"},
    {"scene": "Buscando en el bolso", "action": "doblándose sobre la mesa", "outfit": "falda plisada y blusa", "setting": "entrada de casa", "accessories": "bolso grande"},
    {"scene": "Cambiando ropa", "action": "quitándose una camiseta", "outfit": "sostén de encaje visible", "setting": "dormitorio con espejo", "accessories": "camiseta en mano"},
    {"scene": "Bailando frente al espejo", "action": "ensayando un paso de baile", "outfit": "top corto y shorts", "setting": "habitación con espejo", "accessories": "móvil grabando"},
    {"scene": "Subiendo al auto", "action": "sentándose con falda corta", "outfit": "falda lápiz y tacones", "setting": "estacionamiento", "accessories": "llaves en mano"},
    {"scene": "Tomando selfie", "action": "agachándose para mejor ángulo", "outfit": "vestido ajustado", "setting": "parque con flores", "accessories": "móvil"},
    {"scene": "Limpiando el auto", "action": "inclinándose sobre el capó", "outfit": "shorts vaqueros y top", "setting": "garaje", "accessories": "trapo húmedo"},
    {"scene": "Cocinando descalza", "action": "picando verduras en la encimera", "outfit": "camiseta larga sin nada debajo", "setting": "cocina con isla", "accessories": "cuchillo, tabla"}
]

# --- ESTILOS COMUNES ---
LACE_THONG_STYLES = [
    "delicate black lace thong with floral embroidery",
    "sheer nude illusion thong with lace trim",
    "red satin thong with scalloped edges",
    "white lace thong with pearl detail"
]
HOSIERY_STYLES = [
    "black sheer thigh-high stockings with lace tops",
    "nude ultra-sheer stockings with reinforced toes",
    "fishnet thigh-highs with delicate weave",
    "back-seam stockings with Cuban heel"
]
HEEL_STYLES = [
    "black patent leather stilettos 12cm",
    "nude pumps with red soles",
    "gold strappy sandals",
    "silver chrome stilettos with crystals"
]
CAMERAS = ["Canon EOS R5", "Sony α7R V", "Nikon Z9"]
LENSES = ["85mm f/1.2", "50mm f/1.4"]

# --- CENTRALIZED HISTORY MANAGER ---
class PromptHistoryManager:
    def __init__(self):
        self._history = []

    def add_entry(self, entry):
        self._history.append(entry)

    def get_full_history(self):
        return self._history

    def clear_roles_history(self):
        self._history = [entry for entry in self._history if entry.get("tool") != "Rol Profesional"]
        return self.get_full_history()

    def clear_moments_history(self):
        self._history = [entry for entry in self._history if entry.get("tool") != "Momento Cotidiano"]
        return self.get_full_history()

    def clear_images_history(self):
        self._history = [entry for entry in self._history if entry.get("tool") != "Imagen a Prompt"]
        return self.get_full_history()

    def clear_all_history(self):
        self._history = []
        return self.get_full_history()

# --- GENERADOR DE ROLES ---
class RolePromptGenerator:
    def __init__(self, history_manager):
        self.history_manager = history_manager

    def generate(self, ethnicity, role_name, nsfw_mode):
        role_idx = next((i for i, r in enumerate(ROLES) if r["role"] == role_name), 0)
        role = ROLES[role_idx]
        lace = random.choice(LACE_THONG_STYLES)
        hosiery = random.choice(HOSIERY_STYLES)
        heels = random.choice(HEEL_STYLES)
        camera = random.choice(CAMERAS)
        lens = random.choice(LENSES)

        reveal = "subtle accidental reveal of " + lace if not nsfw_mode else lace + " elegantly visible"
        nudity = "no explicit nudity" if not nsfw_mode else "artistic sensuality permitted"

        prompt = (
            f"Hyper-realistic 9:16 vertical full-body photograph from knee-level low angle, "
            f"of a stunning {ethnicity.lower()} {role['role']} with soft sensual curves, "
            f"wearing {role['uniform']}, {hosiery}, {heels}, "
            f"in {role['environment']}, natural movement causing {reveal}. "
            f"Camera: {camera} with {lens}, professional studio lighting, 16K resolution, "
            f"extreme skin texture with visible pores, subsurface scattering, fabric realism. "
            f"Professional, elegant, subtly sensual — {nudity}. "
            f"Include {role['accessories']} in scene."
        )

        entry = {"tool": "Rol Profesional", "prompt": prompt, "ethnicity": ethnicity, "role": role['role'], "nsfw": nsfw_mode}
        self.history_manager.add_entry(entry)
        return prompt

# --- GENERADOR DE MOMENTOS COTIDIANOS ---
class EverydayMomentGenerator:
    def __init__(self, history_manager):
        self.history_manager = history_manager

    def generate(self, ethnicity, moment_name, nsfw_mode):
        moment_idx = next((i for i, m in enumerate(EVERYDAY_MOMENTS) if m["scene"] == moment_name), 0)
        moment = EVERYDAY_MOMENTS[moment_idx]
        lace = random.choice(LACE_THONG_STYLES)
        hosiery = random.choice(HOSIERY_STYLES) if random.random() > 0.3 else ""
        heels = random.choice(HEEL_STYLES) if any(k in moment["action"] for k in ["tacones", "zapatos", "sentada", "subiendo"]) else ""
        camera = random.choice(CAMERAS)
        lens = random.choice(LENSES)

        reveal = f"accidental y sutil revelación de {lace} al {moment['action']}" if not nsfw_mode else f"{lace} visible de forma natural y sensual durante {moment['action']}"
        hosiery_part = f", {hosiery}" if hosiery else ""
        heels_part = f", {heels}" if heels else ""

        prompt = (
            f"Hyper-realistic 9:16 vertical candid photograph from hidden low angle, "
            f"of a beautiful {ethnicity.lower()} woman in a private everyday moment: {moment['scene'].lower()}, {reveal}. "
            f"Wearing {moment['outfit']}{heels_part}{hosiery_part}, in {moment['setting']}, "
            f"natural body movement, soft morning light, caught in an intimate unguarded instant. "
            f"Camera: {camera} with {lens}, 16K, skin pores, fabric texture, subsurface scattering, cinematic realism. "
            f"Include {moment['accessories']} in scene. "
            f"{'Artistic sensuality' if nsfw_mode else 'No explicit nudity'}."
        )

        entry = {"tool": "Momento Cotidiano", "prompt": prompt, "ethnicity": ethnicity, "scene": moment['scene'], "nsfw": nsfw_mode}
        self.history_manager.add_entry(entry)
        return prompt

# --- IMAGEN A PROMPT ---
class ImageToPromptGenerator:
    def __init__(self, history_manager):
        self.history_manager = history_manager
        self.processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def generate(self, image, ethnicity, nsfw_mode):
        if image is None: return "Sube una imagen."
        try:
            img = Image.fromarray(image.astype('uint8'), 'RGB')
            pixel_values = self.processor(images=img, return_tensors="pt").pixel_values.to(self.device)
            ids = self.model.generate(pixel_values, max_length=120, num_beams=5)
            caption = self.tokenizer.batch_decode(ids, skip_special_tokens=True)[0]

            reveal = "subtle lace thong reveal" if not nsfw_mode else "artistic lace thong visibility"
            nudity = "no explicit nudity" if not nsfw_mode else "tasteful sensuality"

            prompt = (
                f"Hyper-realistic 9:16 vertical low-angle shot inspired by: '{caption}'. "
                f"Transform into elegant {ethnicity} professional woman with {reveal}, "
                f"thigh-high stockings, high heels. Canon EOS R5, 85mm f/1.2, 16K, skin pores, "
                f"fabric realism, cinematic lighting. {nudity}."
            )

            entry = {"tool": "Imagen a Prompt", "prompt": prompt, "ethnicity": ethnicity, "nsfw": nsfw_mode, "source_caption": caption}
            self.history_manager.add_entry(entry)
            return prompt
        except Exception as e:
            return f"Error: {e}"

# --- JS COPIAR ---
copy_js = """(text) => { navigator.clipboard.writeText(text); alert("¡Copiado!"); }"""

# --- EXPORTAR ---
def export_json(hist):
    if not hist: return None, "Sin datos."
    output = io.StringIO(); json.dump(hist, output, indent=2, ensure_ascii=False); output.seek(0)
    return output, "JSON listo"

def export_csv(hist):
    if not hist: return None, "Sin datos."
    output = io.StringIO()
    if not hist: return output, "Sin datos."
    sample = hist[0]
    if "role" in sample:
        fieldnames = ["tool", "prompt", "ethnicity", "role", "nsfw"]
    elif "source_caption" in sample:
        fieldnames = ["tool", "prompt", "ethnicity", "nsfw", "source_caption"]
    else:
        fieldnames = ["tool", "prompt", "ethnicity", "scene", "nsfw"]
    writer = csv.DictWriter(output, fieldnames=fieldnames)
    writer.writeheader(); writer.writerows(hist); output.seek(0)
    return output, "CSV listo"

# --- INSTANCIAS ---
history_manager = PromptHistoryManager()
role_gen = RolePromptGenerator(history_manager)
moment_gen = EverydayMomentGenerator(history_manager)
image_gen = ImageToPromptGenerator(history_manager)

# --- INTERFAZ ---
with gr.Blocks(title="BATUTO ASISTENTE", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# BATUTO ASISTENTE\n**54 Roles · 30 Momentos Cotidianos · Hiperrealismo · NSFW Opcional**")

    with gr.Tabs():
        # === ROLES ===
        with gr.Tab("Roles Profesionales"):
            with gr.Row():
                eth1 = gr.Dropdown(ETHNICITIES, label="Etnia", value="Latina")
                role_dd = gr.Dropdown([r["role"] for r in ROLES], label="Rol", value=ROLES[0]["role"])
                nsfw1 = gr.Checkbox(label="NSFW", value=False)
            out1 = gr.Textbox(label="Prompt", lines=8, interactive=False)
            with gr.Row():
                btn1 = gr.Button("Generar", variant="primary")
                copy1 = gr.Button("Copiar")
            btn1.click(role_gen.generate, [eth1, role_dd, nsfw1], out1)
            copy1.click(fn=None, inputs=out1, js=copy_js)

        # === MOMENTOS ===
        with gr.Tab("Moments Cotidianos"):
            with gr.Row():
                eth2 = gr.Dropdown(ETHNICITIES, label="Etnia", value="Latina")
                moment_dd = gr.Dropdown([m["scene"] for m in EVERYDAY_MOMENTS], label="Momento", value=EVERYDAY_MOMENTS[0]["scene"])
                nsfw2 = gr.Checkbox(label="NSFW", value=False)
            out2 = gr.Textbox(label="Prompt", lines=8, interactive=False)
            with gr.Row():
                btn2 = gr.Button("Generar", variant="primary")
                copy2 = gr.Button("Copiar")
            btn2.click(moment_gen.generate, [eth2, moment_dd, nsfw2], out2)
            copy2.click(fn=None, inputs=out2, js=copy_js)

        # === IMAGEN ===
        with gr.Tab("Imagen a Prompt"):
            img_in = gr.Image(type="numpy", label="Sube una imagen")
            with gr.Row():
                eth3 = gr.Dropdown(ETHNICITIES, value="Latina")
                nsfw3 = gr.Checkbox(value=False)
            out3 = gr.Textbox(lines=8, interactive=False)
            with gr.Row():
                btn3 = gr.Button("Convertir", variant="primary")
                copy3 = gr.Button("Copiar")
            btn3.click(image_gen.generate, [img_in, eth3, nsfw3], out3)
            copy3.click(fn=None, inputs=out3, js=copy_js)

        # === HISTORIAL ===
        with gr.Tab("Historial"):
            hist_json = gr.JSON(label="Historial Completo")
            with gr.Row():
                update_btn = gr.Button("Actualizar")
                clear_role = gr.Button("Borrar Roles", variant="stop")
                clear_moment = gr.Button("Borrar Momentos", variant="stop")
                clear_img = gr.Button("Borrar Imágenes", variant="stop")
                clear_all = gr.Button("Borrar Todo", variant="stop")
                json_btn = gr.Button("Exportar JSON")
                csv_btn = gr.Button("Exportar CSV")

            def get_history_for_display():
                 return history_manager.get_full_history()

            update_btn.click(fn=get_history_for_display, outputs=hist_json)
            clear_role.click(fn=history_manager.clear_roles_history, outputs=hist_json)
            clear_moment.click(fn=history_manager.clear_moments_history, outputs=hist_json)
            clear_img.click(fn=history_manager.clear_images_history, outputs=hist_json)
            clear_all.click(fn=history_manager.clear_all_history, outputs=hist_json)

            json_file = gr.File(); csv_file = gr.File()
            status = gr.Textbox(label="Estado")
            json_btn.click(fn=lambda: export_json(history_manager.get_full_history()), inputs=None, outputs=[json_file, status])
            csv_btn.click(fn=lambda: export_csv(history_manager.get_full_history()), inputs=None, outputs=[csv_file, status])

# --- LANZAR ---
demo.launch(share=True)

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f447fa8212fca91f1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
